In [298]:
# Import libraries
using Turing
using LinearAlgebra
using Distributions
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using StatsBase
using Statistics
using Suppressor
using JLD2
using CSV
using DataFrames, DataFramesMeta
using SplitApplyCombine
using KernelFunctions
using MCMCChains
using PyCall
using PyPlot
using Printf
import PyCall.pyfunction
using Missings

# Access the matplotlib module
matplotlib = pyimport("matplotlib")
seaborn = pyimport("seaborn")
os = pyimport("os")
pyimport("scienceplots")
np = pyimport("numpy")

plt.style.use(["default","science","no-latex"])
using StatsPlots

seaborn.color_palette("colorblind")

PyCall.pygui(:tk)

# Set a seed for reproducibility
using Random
Random.seed!(11);

FONTSIZE=20.5;

In [301]:
function get_up_low(post_dict, upper_quant, lower_quant)
    p = [upper_quant,lower_quant]
    quantiles = zeros(19, 6, 2)
    sorted_keys = sort(collect(keys(post_dict)))
    for i in 1:19
        mat = post_dict[ sorted_keys[i] ]
        quantiles[i, 1,:] = quantile( mat[:,1], p )
        quantiles[i, 2,:] = quantile( mat[:,2], p )
        quantiles[i, 3,:] = quantile( mat[:,3], p )
        quantiles[i, 4,:] = quantile( mat[:,4], p )
        quantiles[i, 5,:] = quantile( mat[:,5], p )
        quantiles[i, 6,:] = quantile( mat[:,6], p )
    end
    return quantiles
end



get_up_low (generic function with 1 method)

In [302]:
Realizations = [string(i) for i in 1:100];
cal_years = collect(range(2015,step=15,length=20));

In [303]:
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
###############################                Parameter Learning Tables             ###########################
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################

In [304]:
# Load the present day posteriors 
present_posterior = np.load("../Data/Training_Data/posterior_samples_All_Combined.npy");
# Take the mean of each parameter
present_bounds = []
present_widths = []
for i in 1:6
    lower = quantile(present_posterior[:,i], 0.05)
    upper = quantile(present_posterior[:,i], 0.95)
    push!(present_bounds, (lower,upper))
    push!(present_widths, upper - lower)
end
present_bounds = reshape(present_bounds, 1, :)
present_widths = reshape(present_widths, 1, :);

In [305]:
# Change to wherever your posterior dictionaries are located
path_to_posteriors = "../Data/Posterior_Data"

# Initialize empty vectors to hold interval bounds and widths 
all_widths = zeros(length(Realizations),19,6)
all_bounds = zeros(length(Realizations), 19, 6, 2)

    for (iter, r) in enumerate(Realizations)
        post = JLD2.load("$(path_to_posteriors)/R_$(r)_Posterior_Dict.jld2", "post_data")
        cred_int_90_up_low = get_up_low(post, 0.95, 0.05)
        all_bounds[iter, :, :, :] = cred_int_90_up_low
        all_widths[iter,:,:] = cred_int_90_up_low[:,:,1] .- cred_int_90_up_low[:,:,2]
    end
# The average 5th percentile and and 95th percentiles of the six MALI parameters across all trajectories over the 19 years of calibration
avg_bounds = mean(all_bounds,dims=1)
avg_bounds = dropdims(avg_bounds, dims=1)

# The average 90% credible interval widths of the six MALI parameters across all trajectories for each of the 19 years of calibration
avg_widths = mean(all_widths,dims=1)
avg_widths = dropdims(avg_widths, dims=1)
# The standard deviation of 90% credible interval widths of the six MALI parameters across all trajectories for each of the 19 years of calibration
std_widths = std(all_widths,dims=1)
println(size(std_widths))
std_widths = dropdims(std_widths,dims=1);

(1, 19, 6)


In [306]:
bounds_as_tuples = fill((0.0,0.0),(19,6))
for i in 1:19
    for j in 1:6
        bounds_as_tuples[i,j] = (r_avg_bounds[i,j,2], r_avg_bounds[i,j,1])
    end
end
#Include the 2015 90% credible interval bounds
bounds_w_present = vcat(present_bounds, bounds_as_tuples);

In [307]:
#Include the 2015 90% credible widths
widths_w_present = vcat(present_widths, avg_widths);

In [308]:
width_change = widths_w_present[2:end,:] .- widths_w_present[1:end-1,:];
#Append the total width change to the end
width_change = vcat(width_change, (widths_w_present[1,:] .- widths_w_present[end,:])');

20×6 Matrix{Float64}:
  -320.73    -0.00491778  -0.0198091    …   -5616.87       -0.127074
    46.0699  -0.00760943  -0.00379009      -12073.2        -0.13987
  -232.377   -0.00177385  -0.00293026      -32593.2        -0.417875
   -80.3277  -0.00430734  -0.00220756      -28763.0        -1.57543
  -311.735   -0.00202516  -0.00484961       -2059.99       -1.21594
  -632.051   -0.00528414  -0.0165426    …   -5534.15       -0.470906
  -277.36    -0.00652996  -0.00591933       -4698.71       -0.184544
  -302.853   -0.00519008  -0.00173952       -2906.17       -1.97547
  -257.523   -0.00378052  -0.003382         -7567.67       -0.806342
  -934.402   -0.00603081  -0.0101035        -4805.13       -1.84227
 -1720.79    -0.0138186   -0.017459     …   -4982.32       -0.408944
  -599.815   -0.0195621   -0.021289         -3871.93       -0.568681
  -355.878   -0.0247099   -0.0169803        -1191.75       -0.460866
 -1248.79    -0.0253337   -0.00923494       -2030.13       -0.339987
  -570.06    -0.0

In [309]:
std_change = std_widths[2:end, :] .- std_widths[1:end-1,:];
#Append the total std change to the end
std_change = vcat(std_change, (std_widths[1, :] .- std_widths[end,:])');

19×6 Matrix{Float64}:
  1450.08     0.0106293     0.0269477    …   22977.3       0.26665
   158.237    0.00129713    0.00152799       -4164.91      0.271446
   215.617    0.00454807    0.0086384        -7243.28      0.857131
    24.8121   0.000603588   0.000754453      -2953.01      0.443092
   -55.5928   0.00246132    0.00830982       -8114.26      0.295317
   315.276    0.00321818    0.00222594   …   -1379.8       0.172519
    70.4188   0.0010485     0.00275042        7684.3      -0.485119
   177.278   -3.75557e-5    0.00152114      -17414.2       0.287537
   -42.5667   0.0020156     0.00191076       -6643.05     -0.188882
   -89.7712   0.000549092   0.00210994       -1947.72     -0.0291988
   393.758    0.00301232   -0.00770615   …    1449.74      0.0187405
   567.646   -0.00502408   -0.00732747        -152.348     0.513363
  -346.908   -0.0153347    -0.00444101       -2725.22     -0.365554
   827.865   -0.00051078    0.00441134         346.051     1.15712
  -868.154   -0.00413269  

In [310]:
# Add the years of calibration as another column
bounds_w_years = hcat(cal_years, bounds_w_present)
widths_w_years = hcat(cal_years, widths_w_present)
std_w_years = hcat(cal_years[2:end], std_widths)
Δ_width_w_years = hcat(vcat(cal_years[2:end], "Total"), width_change)
Δ_std_w_years = hcat(vcat(cal_years[3:end], "Total"), std_change)

names = ["Latest year of calibration","vmThresh","fricExp","mu_scale","stiff_scale","gamma0","melt_flux"]

# Covnert to datframe before saving to disc
df_bounds = DataFrame(bounds_w_years, names)
df_widths = DataFrame(widths_w_years, names)
df_std = DataFrame(std_w_years, names)
df_Δ_width = DataFrame(Δ_width_w_years, names)
df_Δ_std   = DataFrame(Δ_std_w_years, names)

CSV.write("../Data/Parameter_Learning_Tables/param_ninety_percent_bounds.csv",df_bounds)
CSV.write("../Data/Parameter_Learning_Tables/param_ninety_percent_widths.csv",df_widths)
CSV.write("../Data/Parameter_Learning_Tables/param_ninety_percent_std.csv",df_std)
CSV.write("../Data/Parameter_Learning_Tables/param_int_width_change.csv",df_Δ_width)
CSV.write("../Data/Parameter_Learning_Tables/param_int_width_std_change.csv",df_Δ_std)

"../Data/Parameter_Learning_Tables/param_int_width_std_change.csv"

In [ ]:
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
###############################                SLR Projection Learning Tables             ###########################
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################

In [311]:
# All years 2016-2300
all_years_no_gap = collect(range(2016, step=1, length=285))

yrs_dict = Dict{Int64, Int64}()
for (idx,yr) in enumerate(all_years_no_gap)
        yrs_dict[yr] = idx
end

In [312]:
chosen_projection_years = [2100,2200,2300]

all_bounds   = Array{Array{Union{Missing,Tuple{Float64,Float64},Int64}}}(undef, length(chosen_projection_years))
all_widths   = Array{Array{Union{Missing,Float64,Int64}}}(undef, length(chosen_projection_years)) 
all_Δ_widths = Array{Array{Union{Missing,Float64,Int64}}}(undef, length(chosen_projection_years))
all_std      = Array{Array{Union{Missing,Float64,Int64}}}(undef, length(chosen_projection_years))
all_Δ_std    = Array{Array{Union{Missing,Float64,Int64}}}(undef, length(chosen_projection_years))

for (n, year) in enumerate(chosen_projection_years)
    
    proj_yr_idx = yrs_dict[year]
    proj_bounds = Array{Union{Missing,Float64}}(undef,length(Realizations),19,2)
    proj_widths = Array{Union{Missing,Float64}}(undef,length(Realizations),19)
    
    
    for j in 1:length(Realizations)
        for (iter,yr) in enumerate(cal_years[2:end])
                cal_year_matrix = JLD2.load("../Data/Projection_Data/R_$(Realizations[j])/$(Realizations[j])-year$(yr)pred_VAF.jld2",
                    "sample_post_mm_ssp5")
                proj_bounds[j,iter,1] = quantile(cal_year_matrix[proj_yr_idx,:], 0.05)
                proj_bounds[j,iter,2] = quantile(cal_year_matrix[proj_yr_idx,:], 0.95)
                proj_widths[j,iter] = proj_bounds[j,iter,2] - proj_bounds[j,iter,1]
        end
    end

    # Average across the 100 trajectories
    avg_bounds = dropdims(mean(proj_bounds,dims=1),dims=1)
    avg_widths = dropdims(mean(proj_widths,dims=1),dims=1)
    std_widths = dropdims(std(proj_widths,dims=1),dims=1)
    println(typeof(avg_bounds))
    println(typeof(std_widths))

    
    # Load the present day projections
    present_projections = JLD2.load("../Data/Projection_Data/2015_SLR_Projections.jld2","present2015_post_mm_ssp5")
    present_lower = quantile(present_projections[proj_yr_idx,:], 0.05)
    present_upper = quantile(present_projections[proj_yr_idx,:], 0.95)
    present_width = present_upper - present_lower

    # Add present day projections to beginning of lists above
    bounds_w_present = vcat([present_lower;; present_upper], avg_bounds)
    slr_tuple_bounds = [(bounds_w_present[i,1],bounds_w_present[i,2]) for i in 1:length(bounds_w_present[:,1])]
    widths_w_present = vcat([present_width], avg_widths);

    # Calculate year to year change in average cred interval width 
    # As well as the total change from 2015 to 2300
    width_change = vcat(
        widths_w_present[2:end] .- widths_w_present[1:end-1],
        widths_w_present[end] - widths_w_present[1]
    )
    
    # Calculate year to year change in the standard deviation of the 90% cred interval widths
    # As well as the total change from 2030 to 2300
    std_width_change = vcat(
        std_widths[2:end] .- std_widths[1:end-1],
        std_widths[end] - std_widths[1]
    )

    # Push this projection years quantities to list containing all the projection years
    all_bounds[n] = slr_tuple_bounds
    all_widths[n]   =  widths_w_present
    all_Δ_widths[n] = width_change
    all_std[n]      =  std_widths
    all_Δ_std[n]    =  std_width_change
    
    
end


Base.ReshapedArray{Union{Missing, Float64}, 2, Array{Union{Missing, Float64}, 3}, Tuple{}}
Vector{Float64}
.ReshapedArray{Union{Missing, Float64}, 2, Array{Union{Missing, Float64}, 3}, Tuple{}}
Vector{Float64}
ReshapedArray{Union{Missing, Float64}, 2, Array{Union{Missing, Float64}, 3}, Tuple{}}
Vector{Float64}


In [313]:
# Add the years of calibration as another column
bounds_w_years = hcat(cal_years, reduce(hcat, all_bounds))
widths_w_years = hcat(cal_years, reduce(hcat, all_widths))
Δ_width_w_years = hcat( vcat(cal_years[2:end], "Total") , reduce(hcat, all_Δ_widths) )
std_w_years = hcat( cal_years[2:end], reduce(hcat, all_std ) )
Δ_std_w_years = hcat(vcat(cal_years[3:end], "Total"), reduce(hcat, all_Δ_std))


names = ["Latest year of calibration","2100", "2200","2300"]

# Convert to dataframes before saving to disc
df_bounds  = DataFrame(bounds_w_years, names)
df_widths  = DataFrame(widths_w_years, names)
df_std     = DataFrame(std_w_years, names)
df_Δ_width = DataFrame(Δ_width_w_years, names)
df_Δ_std   = DataFrame(Δ_std_w_years, names)

# Remove data that has been calibrated past its projection year (i.e projections for year 2100 with parameters calibrated in 2105)
for yr in names[2:3]
    n_useful = (parse(Int, yr) - 2015) ÷ 15
    df_bounds[n_useful+2:end, yr] .= missing
    df_widths[n_useful+2:end, yr] .= missing 
    df_std[n_useful+1:end, yr] .= missing    
    df_Δ_width[n_useful+1:end, yr] .= missing
    df_Δ_std[n_useful:end, yr] .= missing  
end
    
CSV.write("../Data/SLR_Learning_Tables/SLR_ninety_percent_bounds.csv",df_bounds)
CSV.write("../Data/SLR_Learning_Tables/SLR_ninety_percent_widths.csv",df_widths)
CSV.write("../Data/SLR_Learning_Tables/SLR_ninety_percent_std.csv",df_std)
CSV.write("../Data/SLR_Learning_Tables/SLR_int_width_change.csv",df_Δ_width)
CSV.write("../Data/SLR_Learning_Tables/SLR_int_width_std_change.csv",df_Δ_std)

"../Data/SLR_Learning_Tables/SLR_int_width_std_change.csv"